In [15]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 파일 경로 설정 (폴더 위치를 기준으로 수정)
train_path = "Titanic/train.csv"
test_path = "Titanic/test.csv"
gender_submission_path = "Titanic/gender_submission.csv"

# 각 파일 읽기
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
gender_submission_data = pd.read_csv(gender_submission_path)

# 데이터 확인
print("Train Data:")
print(train_data.head(), "\n")

print("Test Data:")
print(test_data.head(), "\n")

print("Gender Submission Data:")
print(gender_submission_data.head())


Train Data:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN  

타이타닉 생존자 예측을 위해 랜덤 포레스트 알고리즘을 사용하여 승객의 생존 여부를 예측하는 모델을 구축합니다.
1. 데이터 준비
train.csv: 모델을 학습하는 데 사용되는 데이터입니다. 여기에는 승객들의 특성(성별, 나이, 요금 등)과 생존 여부(목표 변수)가 포함됩니다.
test.csv: 예측에 사용할 데이터입니다. 여기에는 승객들의 특성만 포함되고, 생존 여부는 포함되지 않으므로, 우리가 예측한 결과를 제출해야 합니다.

In [16]:
#2. 데이터 전처리
# 데이터 전처리: 성별(Sex)과 탑승 항구(Embarked)를 숫자로 변환
train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1})
test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1})

# 'Embarked'의 결측값을 최빈값으로 채우기
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)
test_data['Embarked'].fillna(test_data['Embarked'].mode()[0], inplace=True)

# 'Fare'와 'Age' 결측값을 평균값으로 채우기
train_data['Fare'].fillna(train_data['Fare'].mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(), inplace=True)


# 범주형 'Embarked' 변환
train_data['Embarked'] = train_data['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
test_data['Embarked'] = test_data['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

성별(Sex): 성별 정보를 남성과 여성을 구분하기 위해 이를 숫자 0 (남성)과 1 (여성)으로 변환합니다.
Embarked 컬럼도 범주형 데이터입니다. 승객들이 탑승한 항구(Cherbourg, Queenstown, Southampton)를 각각 C, Q, S로 나타내며, 이를 숫자로 변환합니다.
결측값 처리: 결측값이 있는 Age, Fare, Embarked 컬럼에 대해 평균값과 최빈값으로 결측값을 채웁니다.

성별(Sex): 성별 정보를 남성과 여성을 구분하기 위해 이를 숫자 0 (남성)과 1 (여성)으로 변환합니다.
Embarked 컬럼도 범주형 데이터입니다. 승객들이 탑승한 항구(Cherbourg, Queenstown, Southampton)를 각각 C, Q, S로 나타내며, 이를 숫자로 변환합니다.
결측값 처리: 결측값이 있는 Age, Fare, Embarked 컬럼에 대해 평균값과 최빈값으로 결측값을 채웁니다.
데이터 전처리: 성별(Sex)과 탑승 항구(Embarked)를 숫자로 변환

In [17]:
# 특징 변수(X)와 목표 변수(y) 설정
X = train_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = train_data['Survived']

# 학습용/검증용 데이터 나누기
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
#3. 모델 학습

# 랜덤 포레스트 모델 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

랜덤 포레스트(Random Forest) 알고리즘을 사용하여 학습합니다.
train.csv의 데이터를 기반으로 승객의 생존 여부(Survived)를 예측하기 위해 여러 개의 결정 트리로 모델을 구성합니다.
RandomForestClassifier(): 랜덤 포레스트 모델을 초기화하고, n_estimators=100으로 100개의 트리를 사용하여 학습합니다.

In [19]:
#4. 성능 평가
#모델을 검증용 데이터(X_val, y_val)로 평가하여 정확도를 계산합니다.
#예측 결과와 실제 생존 여부를 비교하여 모델의 성능을 평가합니다.
# 검증 데이터로 예측
y_pred = rf_model.predict(X_val)


# 성능 평가
accuracy = accuracy_score(y_val, y_pred)
print(f'모델 정확도: {accuracy * 100:.2f}%')

# test.csv 데이터 전처리
test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1})
test_data['Embarked'] = test_data['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
test_data['Age'].fillna(test_data['Age'].mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)

# 특징 변수 설정
X_test = test_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

모델 정확도: 81.01%


모델을 검증용 데이터(X_val, y_val)로 평가하여 정확도를 계산합니다.
예측 결과와 실제 생존 여부를 비교하여 모델의 성능을 평가합니다.

In [20]:
#5. 예측 수행

# 결측값 확인 및 처리
print(X_test.isnull().sum())  # 결측값 확인
X_test.fillna(0, inplace=True)  # 결측값 0으로 채움

# 예측 수행
test_predictions = rf_model.predict(X_test)

# 예측 결과 저장
output = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': test_predictions})
output.to_csv('submission.csv', index=False)

print("Predictions saved to submission.csv!")

Pclass        0
Sex         418
Age           0
SibSp         0
Parch         0
Fare          0
Embarked    418
dtype: int64
Predictions saved to submission.csv!


C:\Users\user\AppData\Local\Temp\ipykernel_11508\294798913.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(0, inplace=True)  # 결측값 0으로 채움


test.csv 데이터를 이용해 승객들의 생존 여부를 예측하고, 결과를 submission.csv 파일로 저장합니다.

In [21]:
#6. 결과 제출

# 결과를 CSV 파일로 저장
output.to_csv('submission.csv', index=False)

print("예측 결과가 'submission.csv' 파일로 저장되었습니다.")

예측 결과가 'submission.csv' 파일로 저장되었습니다.


예측 결과를 Kaggle에 제출할 수 있는 형식으로 저장하여 submission.csv 파일로 출력합니다.